In [ ]:
import sys
import os

# Add DESC to you path (not necessary if you used pip install)
sys.path.insert(0, os.path.abspath("."))
sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../"))

# If you have access to Nvidia GPU
# from desc import set_device
# set_device("gpu")

In [2]:
import numpy as np
np.set_printoptions(linewidth=np.inf, precision=4, suppress=True, threshold=sys.maxsize)
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go

In [3]:
from desc.backend import print_backend_info
from desc.examples import get
from desc.grid import LinearGrid
from desc.plotting import plot_3d

print_backend_info()

DESC version=0.15.0+46.g8fe69f803.
Using JAX backend: jax version=0.6.2, jaxlib version=0.6.2, dtype=float64.
Using device: NVIDIA GeForce RTX 4080 Laptop GPU (id=0), with 9.86 GB available memory.


## Get Equilibrium and Create Grid in $(\rho, \alpha, \zeta)$

In [4]:
eq = get("W7-X")
# create the uniform grid in rho, alpha, zeta coordinates
# if you set a different NFP here, the grid will cover only 2pi/NFP in zeta
grid_raz = LinearGrid(rho=1, theta=10, zeta=1000, NFP=1, sym=False)
# this is an array of shape (N, 3) where N is the total number of grid points
data_raz = grid_raz.nodes

## Find the $(\rho, \theta, \zeta)$ and $(R, \phi, Z)$ equivalents of $(\rho, \alpha, \zeta)$

In [5]:
from desc.equilibrium.coords import map_coordinates

# find the corresponding rho, theta, zeta coordinates
# this is an array of shape (N, 3) where N is the total number of grid points
data_rtz = map_coordinates(
    eq,
    data_raz,
    inbasis=("rho", "alpha", "zeta"),
    outbasis=("rho", "theta", "zeta"),
    maxiter=200,
    guess=data_raz,
)
# find the corresponding R, phi, Z coordinates
# this is an array of shape (N, 3) where N is the total number of grid points
data_rpz = map_coordinates(
    eq,
    data_raz,
    inbasis=("rho", "alpha", "zeta"),
    outbasis=("R", "phi", "Z"),
    maxiter=200,
    guess=data_raz,
)

In [ ]:
# you can save these data as txt files if you want
# np.savetxt("data_raz.txt", data_raz)

## Optionally plot the field lines on $\rho$=1 surface

In [7]:
fig = plot_3d(eq, "|B|")
num_theta = grid_raz.num_theta
for i in range(num_theta):
    fig.add_trace(
        go.Scatter3d(
            x=data_rpz[i::num_theta, 0] * np.cos(data_rpz[i::num_theta, 1]),
            y=data_rpz[i::num_theta, 0] * np.sin(data_rpz[i::num_theta, 1]),
            z=data_rpz[i::num_theta, 2],
            mode="lines",
            showlegend=False,
            marker=dict(size=0),
            hovertext=f"alpha={grid_raz.nodes[i, 1]:.2f}",
            name=f"alpha={grid_raz.nodes[i, 1]:.2f}",
        )
    )
fig

/CODES/DESC/desc/utils.py:572: UserWarning: Unequal number of field periods for grid 1 and basis 5.
  warnings.warn(msg, err)
/CODES/DESC/desc/utils.py:572: UserWarning: Unequal number of field periods for grid 1 and basis 5.
  warnings.warn(msg, err)
